# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution
****

In [1]:
import pandas as pd
import numpy as np
from scipy import stats

In [2]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [3]:
# number of callbacks for black-sounding names
sum(data[data.race=='b'].call)

157.0

In [4]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


<div class="span5 alert alert-success">
<p>Your answers to Q1 and Q2 here</p>
</div>

The most appropriate test for whether race has a significant impact on back is either a classical t-test for the mean, or a bootstrap test for the mean. In either case, we need to conduct a two-sample test, because we are comparing the mean in the sample resumes with white-sounding names with the mean in the sample with black-sounding names.

The assumptions of the CLT are satisfied: The samples are *independent*, because race is randomly assigned to the resumes. The samples are also * identically distributed*, because the resumes are randomly drawn from the same distribution of resumes. 

Now let's look at how many observations of each group we have, so we can determine whether we can rely on asymptotics.

In [5]:
data.race.value_counts()

b    2435
w    2435
Name: race, dtype: int64

Since we have about 2500 observations per group, we can rely on the CLT to yield a normally distributed test-statistic.  This is helpful, since our dependent variable – whether the employer called back or not – is dichotomous, and thus follows a Bernoulli rather than a normal distribution.

The null hypothesis is that the probability that an applicant receives a callback is equally likely for resumes with black- and white-sounding names. I chose a two-sided alternative hypothesis: The chance that an applicant receives a callback is *different* for black- and white-sounding names. Due to our large sample size – about 5000 – our test will have enough power to reach statistical significance even for insubstantial effect sizes anyway; therefore there is no reason to choose a  one-sided alternative hypothesis. In addition, this strategy would also enable us to detect the opposite pattern, namely if many employers actively tried to recruit black candidates to increase the diversity of their workforce.

Let's start by looking at the descriptive statistics:

In [6]:
# Examine mean callback rate
round(data.loc[:, ['race', 'call']].groupby('race').mean(), 3)

,call
race,
b,0.064
w,0.097


This does indeed reveal that white applicants were about 50% more likely to receive a callback (9.7% versus 6.4%).  Let's start by conducting a classical t-test to determine whether this difference could plausibly be due to chance, or whether it likely represents a real relationship:

In [7]:
# Conduct a parametric hypothesis test
np.round_(  
    stats.ttest_ind(data.loc[data.race=='b', 'call'],
                    data.loc[data.race=='w', 'call']), 
    5)

array([-4.11471e+00,  4.00000e-05])

The test yields a p-value of very close to zero (namely 0.004%). This means that race does indeed have a significant impact on whether an applicant receives a callback (because if race did not have an effect, there was virtually a zero chance that we would observe as big of a difference in callbacks as we did).

Now let's conduct a bootstrap hypothesis test to see whether we get a similar result:

In [8]:
def permutation_stat(dataframe, v, group, statistic, rng):
    """
    Create a single permutation of the data, randomly assign groups, 
    and compute test statistic.
    
    Parameters
    ----------
    df : pd.DataFrame
        The data frame where the relevant variables are located.
    v : str
        Name of the variable in the data frame that contains the
        data for which to compute means.
    group : str
        Name of the variable in the data frame that contains a
        categorical variable that identifies the two groups for
        which to compute the test statistic.
    statistic : function
        Function that returns the test statistic of interest.
    rng : np.RandomState instance
        Random number generator.

    Returns
    -------
    numeric
        Value of the test statistic for the permutation sample.
    """

    # Permute values
    v_permuted = rng.permutation(dataframe.loc[:, v])
    
    # Randomly assign groups
    group_1 = v_permuted[: group_value_counts[0]]
    group_2 = v_permuted[group_value_counts[1]:]
    
    # Return test statistic
    stat = statistic(group_1, group_2)
    return(stat)

In [9]:
def p_value(replicates, observed_stat, two_sided=True):
    """
    Computes the p-value of a test statistic using bootstrap replicates.
    
    Parameters:
    -----------
    replicates: pd.Series
        bootstrap replicates (i.e., the value of the test statistic for
        each bootstrap sample.
    observed_stat: numeric
        Value of the test statistic in the actual sample.
    two_sided: bool, optional
        Whether to conduct a one-sided or two-sided hypothesis test.
        Default: True (two-sided).
        
    Returns:
    --------
    numeric
        p-value
    """
    
    # First compute what percentage of bootstrap replicates are LOWER
    p = (replicates < observed_stat).mean() 
    # If more than half are LOWER, compute what percentage is HIGHER 
    # (since we are interested in what proportion is more extreme).
    if p > 0.5:
        p = (replicates > observed_stat).mean()
    
    # If conducting a 2-sided test, adjust the p-value accordingly
    if two_sided == True:
        p = 2 * p
    return(p)


def btest_2samp(df, v, group, statistic, n_permutations=10000):
    """Performs a 2-sample hypothesis test using the bootstrap
    
    Parameters
    ----------
    df: pd.DataFrame
        The data frame where the relevant variables are located.
    v: str
        Name of the variable in the data frame that contains the
        data for which to compute means.
    group: str
        Name of the variable in the data frame that contains a
        categorical variable that identifies the two groups for
        which to compute the test statistic.
    n_permutations: numeric, optional
        Number of bootstrap replicates.  Default: 10000

    Returns
    -------
    numeric
        p-value 
    """

    # Count number of observations in each group
    global group_value_counts # For some reason doesn't work otherwise
    group_value_counts = df.loc[:, group].value_counts()
    
    # Compute actual test statistic
    data_1 = df.loc[df[group]==group_value_counts.index[0], v]
    data_2 = df.loc[df[group]==group_value_counts.index[1], v]
    actual_stat = statistic(data_1, data_2)
    
    # Create random number generator
    rng = np.random.RandomState(3)
    
    # Compute test statistic for each permutation replicate
    permutation_stats = np.empty(shape=n_permutations)
    for replicate in range(n_permutations):
        permutation_stats[replicate] = permutation_stat(df, v, group, 
                                                        statistic, rng)

    # Compute p-value
    return(p_value(permutation_stats, actual_stat))


# Define function to compute difference in means
def diff_in_means(s1, s2):
    """ Returns the difference in means between two pd.Series."""
    return (s1.mean() - s2.mean())


# Conduct 2-sample hypothesis test
print('p-value: ', btest_2samp(df=data, v='call', group='race', 
                    statistic=diff_in_means))

p-value:  0.0002


We get a very similar result: The p-value is very close to 0 (to be precise, it is 0.02%). Thus, we clearly reject the null hypothesis that race doesn't have an impact on callback rate.


<div class="span5 alert alert-success">
<p> Your answers to Q4 and Q5 here </p>
</div>

**Q4:**
It is long been suspected that many employers tend to discriminate against black applicants. In order to prove this, however, it is not enough to rely on anecdotal evidence or even observational data such as unequal representation. Instead, we need to conduct randomized experiments, where the difference between treatment and control group is the race of the applicant. This is commonly done with sending out resumes that have either white- or black-sounding names. 

The data set analyzed here is very comprehensive: It contains data from 5000 such observations, split equally into treatment and control group.  This large sample size allows a precise estimation of the effect of race. Conducting a hypothesis test for equality of the mean yielded a p-value of virtually zero. This means that we can be almost certain that race has an effect whether an applicant receives a callback. In addition to the strong statistical significance, the effect size is also huge: Resumes with white-sounding names were about 50% more likely to get a callback.


**Q5:**
Since my analysis did not control for any other variables, we cannot say how important race is compared to other factors. Because this question is definitely of interest, the next step would be to conduct a regression analysis, where we include the other variables available to us as predictors. This could tell us how important race is relative to other attributes of the applicant and various attributes of the job and employer. In addition, this would also allow us to investigate whether there are any variables that mediate the effect of race. (This could be done by including interaction terms.) For example, it might show that the negative impact of race is particularly high on some kinds of jobs and for some kinds of employers, while this negative effect is ameliorated or even turned positive for workplaces that value a diverse workforce. 

That said, we still can be confident the effect of race is not caused by any confounding variables (i.e., that the effect would decrease if we controlled for other important factors). We know this because race was randomly assigned to the resumes. As a result, it is not possible that the real causal factor is merely another variable correlated with race (e.g., having a high school diploma from a lower-achieving school), because randomization broke up the correlation between race and all other potential factors.